In [1]:
# Import necessary libraries
import os
import sys
import pandas as pd

# Modify sys.path for it to contain the main repo path so we can import modules
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from eda_and_preprocessing.eda_class import EDA
from eda_and_preprocessing.preprocessing_class import Preprocessing

In [2]:
eda = EDA()
eda.print_overview()
eda.check_for_duplicates()

Created dataframe with shape: (3172, 26)

Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3172 entries, 0 to 3171
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Transaction Name               3172 non-null   object 
 1   Transaction Name URL           3172 non-null   object 
 2   Organization Industries        3128 non-null   object 
 3   Lead Investors                 1557 non-null   object 
 4   Investor Names                 2603 non-null   object 
 5   Money Raised                   2208 non-null   float64
 6   Money Raised Currency          2219 non-null   object 
 7   Money Raised (in USD)          2208 non-null   float64
 8   Funding Type                   3172 non-null   object 
 9   Announced Date                 3172 non-null   object 
 10  Pre-Money Valuation            230 non-null    float64
 11  Pre-Money Valuation Currency   230 non-null    o

## After a review of the data we can see the follwing:
1. "Organization Location" consists of City, Region, Country, Continent in the same cell. This has to be splitted up into separate columns 
2. "Organization Industries" and "Investor Names" includes many diffirent values in each cell. These has to be splitted up and put in separate tables. 
3. The following columns contain both numeric values and NaN values. "Money Raised", "Money Raised (in USD)", "Pre-Money Valuation", "Pre-Money Valuation (in USD)", "Total Funding Amount" "Total Funding Amount (in USD)"

## Suggestion on Star Schema

### FactFunding
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Transaction ID (PK)         | Unique identifier for the transaction |
| Organization ID (FK)        | Foreign key referencing DimensionOrganization |
| Money Raised                | Amount of money raised in local currency |
| Money Raised (in USD)       | Amount of money raised in USD |
| Funding Type                | Type of funding (e.g., Seed, Pre-Seed) |
| Announced Date              | Date when the funding was announced |
| Funding Stage               | Stage of funding (e.g., Seed, Series A) |
| Number of Funding Rounds    | Number of funding rounds for the organization |
| Total Funding Amount        | Total amount of funding raised in local currency |
| Total Funding Amount (in USD) | Total amount of funding raised in USD |
| Equity Only                 | Whether it was equity-only funding (Yes/No) |
| Transaction Name            | Name of the transaction            |
| Transaction Name URL        | URL of the transaction             |
| Money Raised Currency       | Currency in which money was raised |
| Pre-Money Valuation         | Valuation before the money was raised |
| Pre-Money Valuation Currency | Currency of the pre-money valuation |
| Pre-Money Valuation (in USD) | Pre-money valuation in USD         |
| Funding Status              | Status of the funding              |
| Number of Investors         | Number of investors involved       |


### DimensionOrganization
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Organization ID (PK)        | Unique identifier for the organization |
| Organization Name           | Name of the organization           |
| Location ID (FK)            | Reference to where the organization is located |
| Organization Name URL            | URL of the organization's Crunchbase profile |
| Organization Description    | Brief description of the organization |
| Organization Website        | Website of the organization        |


### DimensionLocation
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Location ID (PK)            | Unique identifier for the location |
| City                        | City where the organization is located |
| Region                      | Region where the organization is located |
| Country                     | Country where the organization is located |
| Continent                   | Continent where the organization is located |


### DimensionInvestor
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Investor ID (PK)            | Unique identifier for the investor |
| Investor                    | Name of the investor               |

### Investor Mapping Table (many-to-many)
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Transaction ID (FK)            | Unique identifier for the transaction |
| Investor ID (FK)                    | Unique identifier for the investor               |
| IsLeadInvestor                   | Bool which describes who is lead investor in a specific transaction|


### DimensionIndustry
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Industry ID (PK)            | Unique identifier for the industry |
| Industry               | Name of the industry               |

### Industry Mapping Table (many-to-many)
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Organization ID (FK)            | Unique identifier for the transaction |
| Industry ID (FK)                    | Unique identifier for the investor               |

## This star schema will be implemented by the class below. 

In [3]:
preprocessing = Preprocessing(eda.df)

## Check envisioned star schema vs implementation

### FactFunding
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Transaction ID (PK)         | Unique identifier for the transaction |
| Organization ID (FK)        | Foreign key referencing DimensionOrganization |
| Money Raised                | Amount of money raised in local currency |
| Money Raised (in USD)       | Amount of money raised in USD |
| Funding Type                | Type of funding (e.g., Seed, Pre-Seed) |
| Announced Date              | Date when the funding was announced |
| Funding Stage               | Stage of funding (e.g., Seed, Series A) |
| Number of Funding Rounds    | Number of funding rounds for the organization |
| Total Funding Amount        | Total amount of funding raised in local currency |
| Total Funding Amount (in USD) | Total amount of funding raised in USD |
| Equity Only                 | Whether it was equity-only funding (Yes/No) |
| Transaction Name            | Name of the transaction            |
| Transaction Name URL        | URL of the transaction             |
| Money Raised Currency       | Currency in which money was raised |
| Pre-Money Valuation         | Valuation before the money was raised |
| Pre-Money Valuation Currency | Currency of the pre-money valuation |
| Pre-Money Valuation (in USD) | Pre-money valuation in USD         |
| Funding Status              | Status of the funding              |
| Number of Investors         | Number of investors involved       |

In [4]:
preprocessing.df.head()

,Transaction Name,Transaction Name URL,Money Raised,Money Raised Currency,Money Raised (in USD),Funding Type,Announced Date,Pre-Money Valuation,Pre-Money Valuation Currency,Pre-Money Valuation (in USD),Funding Stage,Number of Funding Rounds,Total Funding Amount,Total Funding Amount Currency,Total Funding Amount (in USD),Equity Only Funding,Funding Status,Number of Investors,OrganizationID,TransactionID
0,Seed Round - Flagright,https://www.crunchbase.com/funding_round/flagr...,2800000.0,USD,2800000.0,Seed,2022-07-07,NaN,nan,NaN,Seed,1,2800000.0,USD,2800000.0,True,Seed,11.0,1,1
1,Seed Round - aboutuz,https://www.crunchbase.com/funding_round/about...,632000.0,USD,632000.0,Seed,2022-03-01,NaN,nan,NaN,Seed,1,632000.0,USD,632000.0,True,Seed,1.0,2,2
2,Seed Round - Kubermatic,https://www.crunchbase.com/funding_round/kuber...,NaN,nan,NaN,Seed,2022-04-27,NaN,nan,NaN,Seed,3,8300000.0,USD,8300000.0,True,Seed,1.0,3,3
3,Seed Round - MYNE Homes,https://www.crunchbase.com/funding_round/myne-...,23500000.0,EUR,23938847.0,Seed,2022-07-08,NaN,nan,NaN,Seed,4,63500000.0,EUR,67268844.0,True,Early Stage Venture,15.0,4,4
4,Pre Seed Round - Emulate Energy,https://www.crunchbase.com/funding_round/emula...,21956027.0,SEK,2092864.0,Pre-Seed,2022-07-15,63892307.0,SEK,6090260.0,Seed,3,56561291.0,SEK,5396545.0,True,Seed,4.0,5,5


### DimensionOrganization
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Organization ID (PK)        | Unique identifier for the organization |
| Organization Name           | Name of the organization           |
| Location ID (FK)            | Reference to where the organization is located |
| Organization Name URL            | URL of the organization's Crunchbase profile |
| Organization Description    | Brief description of the organization |
| Organization Website        | Website of the organization        |

In [5]:
preprocessing.organizations_df.head()

,Organization Name,Organization Website,Organization Name URL,Organization Description,OrganizationID,LocationID
0,Flagright,https://flagright.com,https://www.crunchbase.com/organization/flagright,AI-native AML compliance & risk management pla...,1,1
1,aboutuz,https://www.aboutuz.com,https://www.crunchbase.com/organization/aboutuz,aboutuz is the first network for users to safe...,2,2
2,Kubermatic,https://www.kubermatic.com,https://www.crunchbase.com/organization/kuberm...,Kubermatic empowers organizations worldwide to...,3,3
3,MYNE Homes,https://www.myne-homes.de,https://www.crunchbase.com/organization/myne-h...,MYNE Homes is a digital co-ownership platform ...,4,1
4,Emulate Energy,https://www.emulate.energy/,https://www.crunchbase.com/organization/emulat...,Emulate provides SaaS for utilities to offer b...,5,4


### DimensionLocation
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Location ID (PK)            | Unique identifier for the location |
| City                        | City where the organization is located |
| Region                      | Region where the organization is located |
| Country                     | Country where the organization is located |
| Continent                   | Continent where the organization is located |


In [6]:
preprocessing.locations_df.head()

,LocationID,City,Region,Country,Continent
0,1,Berlin,Berlin,Germany,Europe
1,2,Feldkirchen,Bayern,Germany,Europe
2,3,Hamburg,Hamburg,Germany,Europe
3,4,Lund,Skane Lan,Sweden,Europe
4,5,Osnabrück,Niedersachsen,Germany,Europe


### DimensionInvestor
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Investor ID (PK)            | Unique identifier for the investor |
| Investor                    | Name of the investor               |

### Investor Mapping Table (many-to-many)
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Transaction ID (FK)            | Unique identifier for the transaction |
| Investor ID (FK)                    | Unique identifier for the investor               |
| IsLeadInvestor                   | Bool which describes who is lead investor in a specific transaction|


In [7]:
preprocessing.investors_df.head()

,Investor,InvestorID
0,Charles Delingpole,1
1,Donald Bringmann,2
2,Erik Muttersbach,3
3,Four Cities Capital,4
4,Fredrik Thomassen,5


In [8]:
preprocessing.investor_mapping_df.head()

,TransactionID,InvestorID,IsLeadInvestor
0,1,1,False
1,1,2,False
2,1,3,False
3,1,4,False
4,1,5,False


### DimensionIndustry
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Industry ID (PK)            | Unique identifier for the industry |
| Industry               | Name of the industry               |

### Industry Mapping Table (many-to-many)
| Column                     | Description                        |
|----------------------------|------------------------------------|
| Organization ID (FK)            | Unique identifier for the transaction |
| Industry ID (FK)                    | Unique identifier for the investor               |

In [9]:
preprocessing.industries_df.head()

,Industry,IndustryID
0,Compliance,1
1,Financial Services,2
2,FinTech,3
3,Fraud Detection,4
4,Information Technology,5


In [10]:
preprocessing.industry_mapping_df.head()

,OrganizationID,IndustryID
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
